# Modelos de Fusión

- ### Arquitecturas diseñadas para combinar información de diferentes fuentes o tipos de datos, integrando varias representaciones de entrada para mejorar el rendimiento del modelo.
- ### La fusión puede ocurrir en diferentes niveles de la red, ya sea en las capas de entrada, en capas intermedias. 
- ### Estos modelos permiten que las redes neuronales aprendan relaciones más complejas entre los distintos tipos de datos.
- ### Es útil en tareas como la clasificación multimodal (imágenes, audio, texto), la generación de texto o la visión por computadora, donde se combinan distintos tipos de entradas.
- ### También es útil para combinar representaciones distintas de la misma fuente de información.

<img src="figs/fig-modelo_fusion1.png" width="50%">



### Operaciones de fusión

In [ ]:
import torch

# Tensores de ejemplo (3 características de diferentes fuentes)
tensor_a = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)  # (2, 2)
tensor_b = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)  # (2, 2)
tensor_c = torch.tensor([[9, 10], [11, 12]], dtype=torch.float32)  # (2, 2)

# Fusión de tensores por suma
tensor_sum = tensor_a + tensor_b + tensor_c
print("Fusión por suma:")
print(tensor_sum)

# Fusión de tensores por concatenación en la dimensión 1
tensor_concat = torch.cat((tensor_a, tensor_b, tensor_c), dim=1)  # Concatenar columnas
print("\nFusión por concatenación:")
print(tensor_concat)

# Fusión con media
tensor_mean = torch.mean(torch.stack([tensor_a, tensor_b, tensor_c]), dim=0)
print("\nFusión por promedio:")
print(tensor_mean)


### Fusión de representaciones por concatenación de características

<img src="figs/fig-modelo_fusion1.png" width="50%">

</br>
<img src="figs/fig-modelo_fusion2.png" width="50%">


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

# Simular datos de texto
num_samples = 100  # Número de documentos
tfidf_dim = 5500     # Dimensión del vector TF-IDF
embedding_dim = 300  # Dimensión del vector de embeddings

# Generar datos simulados
tfidf_vectors = torch.randn(num_samples, tfidf_dim)  # Vector TF-IDF simulado
embedding_vectors = torch.randn(num_samples, embedding_dim)  # Vector de embeddings simulado

# Fusionar las representaciones (Concatenación)
# Dimensión final: (num_samples, tfidf_dim + embedding_dim)
fused_vectors = torch.cat((tfidf_vectors, embedding_vectors), dim=1)  
print(f"Tamaño del vector fusionado: {fused_vectors.shape}")

# Etiquetas simuladas para clasificación binaria
labels = torch.randint(0, 2, (num_samples,)).long()

# Dividir en conjunto de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split( fused_vectors, labels, test_size=0.2, random_state=42)

# Crear un modelo simple
class FusionSimple(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Inicializar el modelo
# Dimensión del vector fusionado
input_dim = tfidf_dim + embedding_dim  
output_dim = 2  # Clasificación binaria
model = FusionSimple(input_dim, output_dim)

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    print(f"Época {epoch + 1}/{epochs}, Pérdida: {loss.item():.4f}")

# Evaluación
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test)
    y_pred_labels = torch.argmax(y_pred_test, dim=1)
    accuracy = (y_pred_labels == y_test).float().mean()
    print(f"Precisión en prueba: {accuracy:.4f}")


### Red neuronal con el manejo de multiples modalidades o representaciones con fusión intermedia
#### Esta red usa representaciones TFIDF y Embeddings de 300 dimensiones

<img src="figs/fig-modelo_fusion3.png" width="50%">

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

# Simular datos
num_samples = 100  # Número de documentos
tfidf_dim = 5500     # Dimensión del vector TF-IDF
embedding_dim = 300  # Dimensión del vector de embeddings

# Simular vectores TF-IDF y embeddings
tfidf_vectors = torch.randn(num_samples, tfidf_dim)
embedding_vectors = torch.randn(num_samples, embedding_dim)

# Etiquetas simuladas
labels = torch.randint(0, 2, (num_samples,)).long()

# Dividir datos en entrenamiento y prueba


# Dividir los datos de manera consistente para entrenamiento y prueba
X_tfidf_train, X_tfidf_test, X_embed_train, X_embed_test, y_train, y_test = train_test_split( tfidf_vectors, embedding_vectors, labels, test_size=0.2, random_state=42)

# Definir el modelo con fusión en una capa intermedia
class FusionIntermedia(nn.Module):
    def __init__(self, tfidf_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        # Procesamiento inicial de TF-IDF
        self.tfidf_fc = nn.Linear(tfidf_dim, hidden_dim)
        # Procesamiento inicial de embeddings
        self.embedding_fc = nn.Linear(embedding_dim, hidden_dim)
        # Fusión
        self.fusion_fc = nn.Linear(2 * hidden_dim, hidden_dim)
        # Clasificación final
        self.output_fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, tfidf, embedding):
        # Procesar por separado
        tfidf_out = self.tfidf_fc(tfidf)
        tfidf_out = self.relu(tfidf_out)

        embedding_out = self.embedding_fc(embedding)
        embedding_out = self.relu(embedding_out)

        # Fusionar
        fused = torch.cat((tfidf_out, embedding_out), dim=1)
        fused_out = self.fusion_fc(fused)
        fused_out = self.relu(fused_out)
        # Clasificación
        output = self.output_fc(fused_out)
        output = self.sigmoid(fused_out)
        return output

# Inicializar el modelo
hidden_dim = 128  # Dimensión intermedia
output_dim = 2    # Clasificación binaria
model = FusionIntermedia(tfidf_dim, embedding_dim, hidden_dim, output_dim)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_tfidf_train, X_embed_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    print(f"Época {epoch + 1}/{epochs}, Pérdida: {loss.item():.4f}")

# Evaluación
model.eval()
with torch.no_grad():
    y_pred_test = model(X_tfidf_test, X_embed_test)
    y_pred_labels = torch.argmax(y_pred_test, dim=1)
    accuracy = (y_pred_labels == y_test).float().mean()
    print(f"Precisión en prueba: {accuracy:.4f}")


In [ ]:
labels = torch.randint(0, 2, (num_samples,)).long()



### Creación de minibatches para multiples modalidades o representaciones

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Ejemplo de datos de entrada (con tamaño arbitrario para la demostración)
N = 256  # Número de ejemplos
tfidf_dim = 5500
embedding_dim1 = 400
batch_size = 64

# Datos de ejemplo (generados aleatoriamente)
tfidf_data = torch.rand(N, tfidf_dim)
embedding1_data = torch.rand(N, embedding_dim1)
labels = torch.randint(0, 2, (N,))

# Función para crear minibatches
def create_minibatches(X_tfidf, X_emb1, Y, batch_size):
    # Cargar los datos en un dataset de tensores
    dataset = TensorDataset(X_tfidf, X_emb1, Y)
    
    # Crear el DataLoader que divide los datos en minibatches
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return loader

# Crear el DataLoader para los minibatches
loader = create_minibatches(tfidf_data, embedding1_data, labels, batch_size)

# Ejemplo de cómo iterar a través de los minibatches
batch=0
for batch_tfidf, batch_emb1, batch_labels in loader:
    print(f"Batch: {batch}")
    print(f"TF-IDF Batch Shape: {batch_tfidf.shape}")
    print(f"Embedding1 Batch Shape: {batch_emb1.shape}")
    print(f"Labels Batch Shape: {batch_labels.shape}")
    batch+=1

### Incorporación de minibatches para multiples modalidades o representaciones al entrenamiento de la red: TFIDF y Embeddings 300d
<img src="figs/fig-modelo_fusion3.png" width="50%">

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Función para crear minibatches
def create_minibatches(X_tfidf, X_emb1, Y, batch_size):
    # Cargar los datos en un dataset de tensores
    dataset = TensorDataset(X_tfidf, X_emb1, Y)
    
    # Crear el DataLoader que divide los datos en minibatches
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return loader


# Simular datos
num_samples = 100  # Número de documentos
tfidf_dim = 5500     # Dimensión del vector TF-IDF
embedding_dim = 300  # Dimensión del vector de embeddings
batch_size = 64
# Simular vectores TF-IDF y embeddings
tfidf_vectors = torch.randn(num_samples, tfidf_dim)
embedding_vectors = torch.randn(num_samples, embedding_dim)

# Etiquetas simuladas
labels = torch.randint(0, 2, (num_samples,)).long()

# Dividir datos en entrenamiento y prueba


# Dividir los datos de manera consistente para entrenamiento y prueba
# Se envian las dos matrices TFIDF y EMBEDDINGS 
X_tfidf_train, X_tfidf_test, X_embed_train, X_embed_test, y_train, y_test = train_test_split( tfidf_vectors, embedding_vectors, labels, test_size=0.2, random_state=42)

# Crear el DataLoader para los minibatches
loader = create_minibatches(X_tfidf_train, X_embed_train, y_train, batch_size)


# Definir el modelo con fusión en una capa intermedia
class FusionIntermedia(nn.Module):
    def __init__(self, tfidf_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        # Procesamiento inicial de TF-IDF
        self.tfidf_fc = nn.Linear(tfidf_dim, hidden_dim)
        # Procesamiento inicial de embeddings
        self.embedding_fc = nn.Linear(embedding_dim, hidden_dim)
        # Fusión
        self.fusion_fc = nn.Linear(2 * hidden_dim, hidden_dim)
        # Clasificación final
        self.output_fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, tfidf, embedding):
        # Procesar por separado
        tfidf_out = self.relu(self.tfidf_fc(tfidf))
        embedding_out = self.relu(self.embedding_fc(embedding))
        # Fusionar
        fused = torch.cat((tfidf_out, embedding_out), dim=1)
        fused_out = self.relu(self.fusion_fc(fused))
        # Clasificación
        output = self.output_fc(fused_out)
        return output

# Inicializar el modelo
hidden_dim = 128  # Dimensión intermedia
output_dim = 2    # Clasificación binaria
model = FusionIntermedia(tfidf_dim, embedding_dim, hidden_dim, output_dim)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
epochs = 10
for epoch in range(epochs):
    batch=0
    for X_tfidf_train_batch, X_embed_train_batch, y_train_batch in loader:
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_tfidf_train_batch, X_embed_train_batch)
        loss = criterion(y_pred, y_train_batch)
        loss.backward()
        optimizer.step()
        print(f"Batch {batch}  Pérdida: {loss.item():.4f}")
        batch+=1
    print(f"Época {epoch + 1}/{epochs}, Pérdida: {loss.item():.4f}")

# Evaluación
model.eval()
with torch.no_grad():
    y_pred_test = model(X_tfidf_test, X_embed_test)
    y_pred_labels = torch.argmax(y_pred_test, dim=1)
    accuracy = (y_pred_labels == y_test).float().mean()
    print(f"Precisión en prueba: {accuracy:.4f}")


### Forma de los conjuntos de datos

In [ ]:
X_tfidf_train.shape, X_tfidf_test.shape, X_embed_train.shape, X_embed_test.shape, y_train.shape, y_test.shape

### Modelo de multiples modalidades o representaciones al entrenamiento de la red: TFIDF, Embeddings 300d, Embeddings 100d

<img src="figs/fig-modelo_fusion4.png" width="50%">

</br>

<img src="figs/fig-modelo_fusion5.png" width="50%">


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Simular datos
num_samples = 100       # Número de documentos
tfidf_dim = 5500          # Dimensión del vector TF-IDF
embedding_dim1 = 300    # Dimensión del primer vector de embeddings
embedding_dim2 = 100    # Dimensión del segundo vector de embeddings

# Simular vectores TF-IDF y embeddings
tfidf_vectors = torch.randn(num_samples, tfidf_dim)
embedding_vectors1 = torch.randn(num_samples, embedding_dim1)
embedding_vectors2 = torch.randn(num_samples, embedding_dim2)

# Etiquetas simuladas
labels = torch.randint(0, 2, (num_samples,)).long()

# Dividir datos en entrenamiento y prueba
train_size = int(0.8 * num_samples)
X_tfidf_train, X_tfidf_test = tfidf_vectors[:train_size], tfidf_vectors[train_size:]
X_embed1_train, X_embed1_test = embedding_vectors1[:train_size], embedding_vectors1[train_size:]
X_embed2_train, X_embed2_test = embedding_vectors2[:train_size], embedding_vectors2[train_size:]
y_train, y_test = labels[:train_size], labels[train_size:]

# Definir el modelo con tres entradas y fusión en capa intermedia
class MultiplesEntradasFusion(nn.Module):
    def __init__(self, tfidf_dim, embedding_dim1, embedding_dim2, hidden_dim, output_dim):
        super().__init__()
        # Procesamiento inicial de TF-IDF
        self.tfidf_fc = nn.Linear(tfidf_dim, hidden_dim)
        # Procesamiento inicial del primer embedding
        self.embedding1_fc = nn.Linear(embedding_dim1, hidden_dim)
        # Procesamiento inicial del segundo embedding
        self.embedding2_fc = nn.Linear(embedding_dim2, hidden_dim)
        # Fusión
        self.fusion_fc = nn.Linear(3 * hidden_dim, hidden_dim)
        # Clasificación final
        self.output_fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, tfidf, embedding1, embedding2):
        # Procesar cada entrada por separado
        tfidf_out = self.relu(self.tfidf_fc(tfidf))
        embedding1_out = self.relu(self.embedding1_fc(embedding1))
        embedding2_out = self.relu(self.embedding2_fc(embedding2))
        # Fusionar
        fused = torch.cat((tfidf_out, embedding1_out, embedding2_out), dim=1)
        fused_out = self.relu(self.fusion_fc(fused))
        # Clasificación
        output = self.output_fc(fused_out)
        return output

# Inicializar el modelo
hidden_dim = 128  # Dimensión intermedia
output_dim = 2    # Clasificación binaria
model = MultiplesEntradasFusion(tfidf_dim, embedding_dim1, embedding_dim2, hidden_dim, output_dim)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_tfidf_train, X_embed1_train, X_embed2_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    print(f"Época {epoch + 1}/{epochs}, Pérdida: {loss.item():.4f}")

# Evaluación
model.eval()
with torch.no_grad():
    y_pred_test = model(X_tfidf_test, X_embed1_test, X_embed2_test)
    y_pred_labels = torch.argmax(y_pred_test, dim=1)
    accuracy = (y_pred_labels == y_test).float().mean()
    print(f"Precisión en prueba: {accuracy:.4f}")


# Ejercicios



# Ejercicio 1 
- ## Para la arquitectura MultiplesEntradasFusion: 
    ### 1. Crear los minibatches para organizar la separación de datos adecuadamente
    ### 1.1. Usar 512 Ejemplos de entrenamiento
    ### 1.1. Usar un batch_size = 32


# Ejercicio 2 
- ## Para la arquitectura de red sobre el conjunto de datos de agresividad: 
    ### 1. Aplicar el modelo de fusión intermedia
    - ### 1.1. Para la subred de TFIDF aplicar 3 capas ocultas
    - ### 1.2. Para la subred de Embeddings de 300d aplicar 2 capas ocultas
    - ### 1.3. Fusionar las salidas de 1.1 y 1.3, para posteriormente aplicar otra subred de 2 capas ocultas
    - ### 1.4. Aplicar una capa de salida
    ### 2. Evaluar el modelo



# Ejercicio 3
- ## Para la arquitectura del ejercicio 2
    ### 1. Aplicar regularización en las diferentes capas ocultas (dropout)
    ### 2. Evaluar el modelo 